# Calling SARS-CoV-2 variants from a metagenomic profiling sample

In this assignment, we are going to call SARS-CoV-2 variants from a metagenomic profiling sample.

To call variants, we need to compare it to a wild-type reference sequence, which is usually the reference genome, but where does these reference genome came from?

There are several ways of doing it. The reference genome for SARS-CoV-2 was generated as follows ([reference](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7094943/)):

1. Patient with pneumonia of unknown origin were examined, and bronchioaveolar lavage fluid was collected (from washing the respiratory tract with saline).

2. Assuming the RNA extraction, RT-PCR, and library construction was performed followed by deep RNA-Seq (56 million reads).

3. Assuming that when we sequence deep enough, every read will have at least one other read that partially overlaps with it, so in theory, we can assemble the whole transcriptome by carefully concatenating reads according to the overlap (For details, check: de novo assembly).


The data we are analyzing is also RNA-seq but is a pool of nasal swab samples, so it is expected to sample different variants that are endemic in the region where samples are collected.

## Step 0: If you are interested in how we retrieve that data from GEO datasets

The data we are analyzing is public and retrieved from https://www.ncbi.nlm.nih.gov/sra/SRR11801823. To retrieve sequence file from GEO dataset, `sra-tools` provides easy access.

In [ ]:
cd data/raw_seq

# fastq-dump downloads data in fastq format
## --split-3 forces sra-tools to check if the data is paired-ended or not.
## The 3 comes from the fact that some sequencing experiments splits read in 3 proportions.
## For example, single-cell RNA-seq using 10X Genomics Chromium platform are sequenced in a paired-ended mode 
## and gives read 1 and read 2, but there's an additional index read.
## `--split-3` makes sure these reads are downloaded separatedly.
fastq-dump --split-3 SRR11801823

## Step 1: Check the adapter database

In [1]:
head Sequencing_adaptors.fasta

>Illumina Single End Apapter 1					
ACACTCTTTCCCTACACGACGCTGTTCCATCT
>Illumina Single End Apapter 2					
CAAGCAGAAGACGGCATACGAGCTCTTCCGATCT
>Illumina Single End PCR Primer 1				
AATGATACGGCGACCACCGAGATCTACACTCTTTCCCTACACGACGCTCTTCCGATCT
>Illumina Single End PCR Primer 2				
CAAGCAGAAGACGGCATACGAGCTCTTCCGATCT
>Illumina Single End Sequencing Primer			
ACACTCTTTCCCTACACGACGCTCTTCCGATCT


Fasta files is a simple format in which there are header lines (starting with >) containing metadata/information about a sequence, and the sequence in the next line. If you were to add a new entry into the file, just open the file with a text editor, and add the following at the bottom of the file:

```
>Name of my sequence
[Sequence]
```

Adapter files that trimming tools (e.g., Trimmomatic, Cutadapt...) use are usually fasta files containing the known adapters. The tools will use the adapter sequence and the threshold provided to find and remove the seqeunces.

Note that it is not advisable to use a comprehensive adapter database. Instead, we should always know what adapters (corresponding to the library generation kit) we are using, and this information should be reported when we publish the study and deposit the data in a public database like GEO Profile.

Therefore, if we know we are working with an experiment using TruSeqv3 kit, we should only trim with adpater sequence corresponding to TruSeqv3.

In [9]:
# -A asks grep to also print the line *A*fter the match is found
grep "Illumina.*Sequencing Primer 1" Sequencing_adaptors.fasta -A 1

>Illumina Paried End Sequencing Primer 1			
ACACTCTTTCCCTACACGACGCTCTTCCGATCT


## Step 2: Perform FastQC

In [12]:
module purge
module load fastqc/0.11.9

# Create a folder to store the output

mkdir fastqc
fastqc ./data/raw_seq/SRR11801823_1.fastq \
       ./data/raw_seq/SRR11801823_2.fastq \
       -o fastqc

Started analysis of SRR11801823_1.fastq
Approx 5% complete for SRR11801823_1.fastq
Approx 10% complete for SRR11801823_1.fastq
Approx 15% complete for SRR11801823_1.fastq
Approx 20% complete for SRR11801823_1.fastq
Approx 25% complete for SRR11801823_1.fastq
Approx 30% complete for SRR11801823_1.fastq
Approx 35% complete for SRR11801823_1.fastq
Approx 40% complete for SRR11801823_1.fastq
Approx 45% complete for SRR11801823_1.fastq
Approx 50% complete for SRR11801823_1.fastq
Approx 55% complete for SRR11801823_1.fastq
Approx 60% complete for SRR11801823_1.fastq
Approx 65% complete for SRR11801823_1.fastq
Approx 70% complete for SRR11801823_1.fastq
Approx 75% complete for SRR11801823_1.fastq
Approx 80% complete for SRR11801823_1.fastq
Approx 85% complete for SRR11801823_1.fastq
Approx 90% complete for SRR11801823_1.fastq
Approx 95% complete for SRR11801823_1.fastq
Analysis complete for SRR11801823_1.fastq
Started analysis of SRR11801823_2.fastq
Approx 5% complete for SRR11801823_2.fastq


## Step 3: Trimming adapters and low-quality bases

We do see adapter enrichment in FastQC report. It is worth noting that though FastQC detects most commonly used adapters, the adpater from your kit might not be in their database.

The data we are analyzing is constructed with an NEB kit, so we will add adapter information for NEB to our adapter database.

In [15]:
# Add NEB adapter data
cat NEB_adapt.fa

>NEB Universal adapter 1
AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC
>NEB Universal adapter 2
AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTAGATCTCGGTGGTCGCCGTATCATT

In [16]:
cat NEB_adapt.fa >> Sequencing_adaptors.fasta

In [17]:
module load trimmomatic/0.39

mkdir int
java -jar $TRIMMOMATIC_JAR PE -phred33 \
data/raw_seq/SRR11801823_1.fastq data/raw_seq/SRR11801823_2.fastq \
int/read_1_trimmed_n.fq int/read_1_unpair_trimmed_n.fq \
int/read_2_trimmed_n.fq int/read_2_unpair_trimmed_n.fq \
ILLUMINACLIP:Sequencing_adaptors.fasta:2:30:10:2:keepBothReads TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:100

mkdir: cannot create directory ‘int’: File exists
TrimmomaticPE: Started with arguments:
 -phred33 data/raw_seq/SRR11801823_1.fastq data/raw_seq/SRR11801823_2.fastq int/read_1_trimmed_n.fq int/read_1_unpair_trimmed_n.fq int/read_2_trimmed_n.fq int/read_2_unpair_trimmed_n.fq ILLUMINACLIP:Sequencing_adaptors.fasta:2:30:10:2:keepBothReads TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:100
Multiple cores found: Using 4 threads
Using Long Clipping Sequence: 'CAAGCAGAAGACGGCATACGAGATTGCCGAGTGACTGGAGTTCCTTGGCACCCGAGAATTCCA'
Using Long Clipping Sequence: 'CTTACTCCTTGGAGGCCATG>NEB Universal adapter 1AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTAGATCTCGGTGGTCGCCGTATCATT'
Using Long Clipping Sequence: 'GATCGGAAGAGCACACGTCTGAACTCCAGTCACCTTGTAATCTCGTATGCCGTCTTCTGCTTG'
Using Long Clipping Sequence: 'AATGATACGGCGACCACCGAGATCTACACGTTCAGAGTTCTACAGTCCGA'
ILLUMINACLIP: Using 0 prefix pairs, 5 forward/reverse sequences, 0 forward only sequences, 0 reverse only 

In [18]:
# And we run fastqc again
fastqc int/read_1_trimmed_n.fq \
       int/read_2_trimmed_n.fq \
       -o fastqc

Started analysis of read_1_trimmed_n.fq
Approx 5% complete for read_1_trimmed_n.fq
Approx 10% complete for read_1_trimmed_n.fq
Approx 15% complete for read_1_trimmed_n.fq
Approx 20% complete for read_1_trimmed_n.fq
Approx 25% complete for read_1_trimmed_n.fq
Approx 30% complete for read_1_trimmed_n.fq
Approx 35% complete for read_1_trimmed_n.fq
Approx 40% complete for read_1_trimmed_n.fq
Approx 45% complete for read_1_trimmed_n.fq
Approx 50% complete for read_1_trimmed_n.fq
Approx 55% complete for read_1_trimmed_n.fq
Approx 60% complete for read_1_trimmed_n.fq
Approx 65% complete for read_1_trimmed_n.fq
Approx 70% complete for read_1_trimmed_n.fq
Approx 75% complete for read_1_trimmed_n.fq
Approx 80% complete for read_1_trimmed_n.fq
Approx 85% complete for read_1_trimmed_n.fq
Approx 90% complete for read_1_trimmed_n.fq
Approx 95% complete for read_1_trimmed_n.fq
Analysis complete for read_1_trimmed_n.fq
Started analysis of read_2_trimmed_n.fq
Approx 5% complete for read_2_trimmed_n.fq


Note from the fastqc report that before trimming, the length of every read is 151 bp, while after trimming, the range becomes 100 - 151, and adapter enrichment at the 3' end is gone.

## Step 4: Align to the genome

In [19]:
module load bwa/intel/0.7.17

# For the first time running it, we'll need to index the genome
cd data/genome
bwa index NC_045512.2.fa

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index NC_045512.2.fa
[main] Real time: 0.406 sec; CPU: 0.016 sec


In [26]:
cd ../..
mkdir result
# Align the reads
bwa mem data/genome/NC_045512.2.fa \
        int/read_1_trimmed_n.fq \
        int/read_2_trimmed_n.fq > result/aligned_reads.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 66472 sequences (10000123 bp)...
[M::process] read 66440 sequences (10000141 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (49, 33004, 68, 46)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (76, 87, 135)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 253)
[M::mem_pestat] mean and std.dev: (94.40, 33.66)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 312)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (148, 197, 261)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 487)
[M::mem_pestat] mean and std.dev: (214.44, 79.91)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 600)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 50, 75) percentile: (

## Step 5: Convert to BAM and sort by coordinate

In [27]:
module load picard/2.23.8

java -jar $PICARD_JAR SortSam \
          INPUT=result/aligned_reads.sam \
          OUTPUT=result/sorted_reads.bam \
          SORT_ORDER=coordinate

INFO	2021-03-24 14:06:29	SortSam	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    SortSam -INPUT result/aligned_reads.sam -OUTPUT result/sorted_reads.bam -SORT_ORDER coordinate
**********


14:06:29.395 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/picard/2.23.8/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Mar 24 14:06:29 EDT 2021] SortSam INPUT=result/aligned_reads.sam OUTPUT=result/sorted_reads.bam SORT_ORDER=coordinate    VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json USE_JDK_DEFLATER=false USE_JDK_INFLATER=false
[Wed M

## Step 6: Add read group information

In [28]:
## Add read group to the bam file
## The platform and model information, while not relevant for this analysis,
## can be found at the information page on GEO dataset
java -jar $PICARD_JAR AddOrReplaceReadGroups \
          I=result/sorted_reads.bam \
          O=result/sorted_reads_rg.bam \
          RGID=SRR11801823 \
          RGLB=SRR11801823 \
          RGPL=ILLUMINA \
          RGPM=iSEQ \
          RGPU=SRR11801823 \
          RGSM=SRR11801823

INFO	2021-03-24 14:08:04	AddOrReplaceReadGroups	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    AddOrReplaceReadGroups -I result/sorted_reads.bam -O result/sorted_reads_rg.bam -RGID SRR11801823 -RGLB SRR11801823 -RGPL ILLUMINA -RGPM iSEQ -RGPU SRR11801823 -RGSM SRR11801823
**********


14:08:04.556 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/picard/2.23.8/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Mar 24 14:08:04 EDT 2021] AddOrReplaceReadGroups INPUT=result/sorted_reads.bam OUTPUT=result/sorted_reads_rg.bam RGID=SRR11801823 RGLB=SRR11801823 RGPL=ILLUMINA RGPU=SRR11801823 RGSM=SRR11801823 RGPM=iSEQ    VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=ST

## Step 7: Mark duplicated reads

In [30]:
java -jar $PICARD_JAR MarkDuplicates \
          INPUT=result/sorted_reads_rg.bam \
          OUTPUT=result/dedup_reads.bam \
          METRICS_FILE=metrics.txt

INFO	2021-03-24 14:08:39	MarkDuplicates	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    MarkDuplicates -INPUT result/sorted_reads_rg.bam -OUTPUT result/dedup_reads.bam -METRICS_FILE metrics.txt
**********


14:08:39.955 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/picard/2.23.8/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Mar 24 14:08:39 EDT 2021] MarkDuplicates INPUT=[result/sorted_reads_rg.bam] OUTPUT=result/dedup_reads.bam METRICS_FILE=metrics.txt    MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP=50000 MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=8000 SORTING_COLLECTION_SIZE_RATIO=0.25 TAG_DUPLICATE_SET_MEMBERS=false REMOVE_SEQUENCING_DUPLICATES=false TAGGING_POLICY=DontT

MarkDuplicate mode of picard utilizes several information to find possible PCR duplicates: It examines the 5' end of the read, and also try to infer duplicates generated in clutser formation in Illumina sequencing.

The coordinate of a read in the sequencing lane will be stored in the read names in resulting fastq files, but GEO dataset (or `sra-tools`) does not preserve this information, as a result, when picard tries to learn coordinates from read names, it struggles and produces warnings like:

```
WARNING	2021-03-24 14:08:40	AbstractOpticalDuplicateFinderCommandLineProgram	A field field parsed out of a read name was expected to contain an integer and did not. Read name: SRR11801823.111780. Cause: String 'SRR11801823.111780' did not start with a parsable number.
```

It is safe to ingore the warning.

## Step 8: Prepare reference dictionary, fasta index, and bam index

To make sure the aligned reads (BAM file) are aligned to the same reference genome you are using to call variants, the reference genome has to be made a dictionary.

Under the table, when fastq files are aligned, the resulting SAM/BAM files contain the version of reference genome in its header.

Dictionary files of a reference genome is just extracting version information and save it in SAM header format for the ease of comparison.

Indexing the reference genome fasta and the BAM file is to facilitate quick access to them. Indexing is conceptually like providing a table of content.

If you don't index a sorted BAM file, when you need to find a read coming from chr17, you'll need to scroll down from chr1, because you don't know exactly how many lines you have to skip to get to chr17. After indexing, information like "chr17 starts from the kth line" are stored in the index file, and now other tools can access the fasta/BAM file with a higher efficiency.

In [31]:
java -jar $PICARD_JAR CreateSequenceDictionary \
          R=data/genome/NC_045512.2.fa \
          O=data/genome/NC_045512.2.dict

INFO	2021-03-24 14:17:00	CreateSequenceDictionary	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    CreateSequenceDictionary -R data/genome/NC_045512.2.fa -O data/genome/NC_045512.2.dict
**********


14:17:00.231 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/picard/2.23.8/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Mar 24 14:17:00 EDT 2021] CreateSequenceDictionary OUTPUT=data/genome/NC_045512.2.dict REFERENCE=data/genome/NC_045512.2.fa    TRUNCATE_NAMES_AT_WHITESPACE=true NUM_SEQUENCES=2147483647 VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRET

In [32]:
# A dictionary is just a SAM header
cat data/genome/NC_045512.2.dict

@HD	VN:1.6
@SQ	SN:NC_045512.2	LN:29903	M5:105c82802b67521950854a851fc6eefd	UR:file:/scratch/ycc520/ag_recitation/asm_03_key/data/genome/NC_045512.2.fa


In [33]:
# Indexing with samtools
module load samtools/intel/1.11

samtools faidx data/genome/NC_045512.2.fa
samtools index result/dedup_reads.bam

## Step 9: Call haplotypes with GATK

In [34]:
module load gatk/4.1.9.0

java -jar $GATK_JAR HaplotypeCaller \
          -R data/genome/NC_045512.2.fa \
          -I result/dedup_reads.bam \
          -O result/raw_variants.vcf

14:19:03.177 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Mar 24, 2021 2:19:03 PM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
14:19:03.304 INFO  HaplotypeCaller - ------------------------------------------------------------
14:19:03.305 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.1.9.0
14:19:03.305 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
14:19:03.305 INFO  HaplotypeCaller - Executing as ycc520@cs053.nyu.cluster on Linux v4.18.0-193.28.1.el8_2.x86_64 amd64
14:19:03.305 INFO  HaplotypeCaller - Java runtime: Java HotSpot(TM) 64-Bit Server VM v1.8.0_271-b09
14:19:03.305 INFO  HaplotypeCaller - Start Date/Time: March 24, 2021 2:19:03 PM EDT
14:19:03.305 INFO  HaplotypeCalle

## Step 10: Split the variants to snps and indels and filter SNPs by quality/calling confidence

In [36]:
gatk SelectVariants \
     -R data/genome/NC_045512.2.fa \
     -V result/raw_variants.vcf \
     -select-type SNP \
     -O result/raw_snps.vcf
     
gatk SelectVariants \
     -R data/genome/NC_045512.2.fa \
     -V result/raw_variants.vcf \
     -select-type INDEL \
     -O result/raw_indels.vcf

Using GATK jar /share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar defined in environment variable GATK_LOCAL_JAR
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar SelectVariants -R data/genome/NC_045512.2.fa -V result/raw_variants.vcf -select-type SNP -O result/raw_snps.vcf
14:24:16.684 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Mar 24, 2021 2:24:16 PM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
14:24:16.800 INFO  SelectVariants - ------------------------------------------------------------
14:24:16.800 INFO  SelectVariants - The Genome Analysis Toolkit (GATK)

In [38]:
# Filter SNPs with quality measures

java -jar $GATK_JAR VariantFiltration \
          -R data/genome/NC_045512.2.fa \
          -V result/raw_snps.vcf \
          --filter-name "QD_filter" \
          -filter "QD<2.0" \
          --filter-name "FS_filter" \
          -filter "FS>60.0" \
          --filter-name "MQ_filter" \
          -filter "MQ<40.0" \
          --filter-name "SOR_filter" \
          -filter "SOR>10.0" \
          -O result/filtered_snps.vcf

14:25:07.637 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Mar 24, 2021 2:25:07 PM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
14:25:07.757 INFO  VariantFiltration - ------------------------------------------------------------
14:25:07.757 INFO  VariantFiltration - The Genome Analysis Toolkit (GATK) v4.1.9.0
14:25:07.757 INFO  VariantFiltration - For support and documentation go to https://software.broadinstitute.org/gatk/
14:25:07.757 INFO  VariantFiltration - Executing as ycc520@cs053.nyu.cluster on Linux v4.18.0-193.28.1.el8_2.x86_64 amd64
14:25:07.757 INFO  VariantFiltration - Java runtime: Java HotSpot(TM) 64-Bit Server VM v1.8.0_271-b09
14:25:07.757 INFO  VariantFiltration - Start Date/Time: March 24, 2021 2:25:07 PM EDT
14:25:07.757 INFO  Va

## Step 11: Annotate the filtered SNPs with snpEff

snpEff annotates snps and indels by incorporating the information from a gene model. This allows us to learn whether a mutation is within a gene or in the intergenic region, and if a mutations causes the amino acid sequence of a gene to change or terminate prematurely.

snpEff has annotation database generated for common organisms but not for SARS-CoV-2. The database was thus manually curated and provided for the assignment. I will show how the annotation is generated below and annotate the filtered snp list.

In [39]:
# To add a custom annotation, first we need to give it a name in the config file.
cat snpEff.config

# SARS-CoV-2 genome, version GCF_009858895.2
sarscov2.genome : SARS_COV_2


The two files are downloaded and renamed as `sequences.fa` and `genes.gff` respectively in the folder name defined in `snpEff.config`.

If the genome is defined as sars.cov2.genome, the two files should be stored in data/sarscov2. 

In [40]:
tree data

data
├── genome
│   ├── NC_045512.2.dict
│   ├── NC_045512.2.fa
│   ├── NC_045512.2.fa.amb
│   ├── NC_045512.2.fa.ann
│   ├── NC_045512.2.fa.bwt
│   ├── NC_045512.2.fa.fai
│   ├── NC_045512.2.fa.pac
│   └── NC_045512.2.fa.sa
├── raw_seq
│   ├── SRR11801823_1.fastq
│   └── SRR11801823_2.fastq
└── sarscov2
    ├── genes.gff
    └── sequences.fa

3 directories, 12 files


In [41]:
# Then, the annotation database is generated from the refernce genome (fasta) and gene model (gff/gtf)
module load snpeff/4.3t

# -v points to where you put the .fa and .gff files under data/, in our case, it's sarscov2.
java -jar $SNPEFF_JAR build -gff3 -v sarscov2

00:00:00	SnpEff version SnpEff 4.3t (build 2017-11-24 10:18), by Pablo Cingolani
00:00:00	Command: 'build'
00:00:00	Building database for 'sarscov2'
00:00:00	Reading configuration file 'snpEff.config'. Genome: 'sarscov2'
00:00:00	Reading config file: /scratch/ycc520/ag_recitation/asm_03_key/snpEff.config
00:00:00	done
Reading GFF3 data file  : '/scratch/ycc520/ag_recitation/asm_03_key/./data/sarscov2/genes.gff'

	Total: 11 markers added.

	Create exons from CDS (if needed): .............
	Exons created for 11 transcripts.

	Deleting redundant exons (if needed): .
		0	
		Total transcripts with deleted exons: 1

	Collapsing zero length introns (if needed): .
		0	
		Total collapsed transcripts: 1
	Reading sequences   :
	FASTA file: '/scratch/ycc520/ag_recitation/asm_03_key/./data/genomes/sarscov2.fa' not found.
	Reading FASTA file: '/scratch/ycc520/ag_recitation/asm_03_key/./data/sarscov2/sequences.fa'
		Reading sequence 'NC_045512.2', length: 29903
		Adding genomic sequences to exons: 	D

In [42]:
# Now we can annotate the snps with the newly generated database

java -jar $SNPEFF_JAR \
     -v sarscov2 \
     result/filtered_snps.vcf > result/filtered_snps.ann.vcf

00:00:00	SnpEff version SnpEff 4.3t (build 2017-11-24 10:18), by Pablo Cingolani
00:00:00	Command: 'ann'
00:00:00	Reading configuration file 'snpEff.config'. Genome: 'sarscov2'
00:00:00	Reading config file: /scratch/ycc520/ag_recitation/asm_03_key/snpEff.config
00:00:00	done
00:00:00	Reading database for genome version 'sarscov2' from file '/scratch/ycc520/ag_recitation/asm_03_key/./data/sarscov2/snpEffectPredictor.bin' (this might take a while)
00:00:00	done
00:00:00	Loading Motifs and PWMs
00:00:00	Building interval forest
00:00:00	done.
00:00:00	Genome stats :
#-----------------------------------------------
# Genome name                : 'SARS_COV_2'
# Genome version             : 'sarscov2'
# Genome ID                  : 'sarscov2[0]'
# Has protein coding info    : true
# Has Tr. Support Level info : true
# Genes                      : 11
# Protein coding genes       : 11
#-----------------------------------------------
# Transcripts                : 11
# Avg. transcripts per gene

### Q1: (15%) Which gene has the most filtered SNPs? (Hint: snpEff provides a gene summary table)

You can open the `snpEff_genes.txt` with Excel or R for better readability. It is worth noting that `variant_impact` and `variant_effect_*` could overlap: A missense variant could have moderate impact, so if you simply take the row mean, it will be counted twice.

Personally, I would count only the columns for `variant_effect`: They account for mutations upstream and down stream of a gene, and mutations within a gene (synonymous or missense).

You can also visualize the SNPs in IGV and count manually, but this will not be possible for a profiling of larger scale where you might have hundreds of SNPs in a gene.

In [43]:
head snpEff_genes.txt

# The following table is formatted as tab separated values. 
#GeneName	GeneId	TranscriptId	BioType	variants_impact_LOW	variants_impact_MODERATE	variants_impact_MODIFIER	variants_effect_downstream_gene_variant	variants_effect_missense_variant	variants_effect_synonymous_variant	variants_effect_upstream_gene_variant
E	gene-GU280_gp04	TRANSCRIPT_gene-GU280_gp04	protein_coding	0	0	4	3	0	0	1
M	gene-GU280_gp05	TRANSCRIPT_gene-GU280_gp05	protein_coding	0	0	4	3	0	0	1
N	gene-GU280_gp10	TRANSCRIPT_gene-GU280_gp10	protein_coding	1	2	1	0	2	1	1
ORF10	gene-GU280_gp11	TRANSCRIPT_gene-GU280_gp11	protein_coding	0	0	3	0	0	0	3
ORF1ab	gene-GU280_gp01	TRANSCRIPT_gene-GU280_gp01	protein_coding	3	7	2	1	7	3	1
ORF3a	gene-GU280_gp03	TRANSCRIPT_gene-GU280_gp03	protein_coding	0	0	4	3	0	0	1
ORF6	gene-GU280_gp06	TRANSCRIPT_gene-GU280_gp06	protein_coding	0	0	4	3	0	0	1
ORF7a	gene-GU280_gp07	TRANSCRIPT_gene-GU280_gp07	protein_coding	0	0	4	3	0	0	1


### Q2: (15%) Is there any missense mutation on the spike protein (gene S)? If there is, what is the amino acid change? Please answer in the format of original AA - position - mutated AA (For example, the change of the 93th amino acid from Glycine to Alanine is G93A).

`vcf_summary.sh` helps you to format the VCF file so you can see the mutations and there corresponding genes.

In [46]:
./vcf_summary.sh result/filtered_snps.ann.vcf | grep miss

T  missense_variant         MODERATE  ORF1ab            p.Ile300Phe
G  missense_variant         MODERATE  ORF1ab            p.Ser1218Gly
C  missense_variant         MODERATE  ORF1ab            p.Ile2057Thr
T  missense_variant         MODERATE  ORF1ab            p.Gln2058His
A  missense_variant         MODERATE  ORF1ab            p.Met4766Ile
T  missense_variant         MODERATE  ORF1ab            p.Ser5585Ile
C  missense_variant         MODERATE  ORF1ab            p.Leu5624Pro
G  missense_variant         MODERATE  S                 p.Asp614Gly
A  missense_variant         MODERATE  N                 p.Arg203Lys
C  missense_variant         MODERATE  N                 p.Gly204Arg


Here, we can clearly see that D614G mutation is present in the spike protein. If you search for this mutation, you will find that this mutation soon become very prevalent globally because it is associated with higher transmissibility, presumably by reducing spike protein shedding and enhanced incorporation of spike proteins to the virion [[1]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7310631/).